In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_path = "../input/dogs-vs-cats/test1.zip"
test_path = "../input/dogs-vs-cats/train.zip"

files = "/kaggle/working/"

import zipfile

with zipfile.ZipFile(train_path, 'r') as zipp:
    zipp.extractall(files)
    
with zipfile.ZipFile(test_path, 'r') as zipp:
    zipp.extractall(files)

In [ ]:
image_dir = "../working/train/"

filenames = os.listdir(image_dir)

labels = [x.split(".")[0] for x in filenames]

data = pd.DataFrame({
     "filename": filenames, 
     "label": labels})
data.head()

image_dir = "../working/test1/"

filenames = os.listdir(image_dir)

labeltest = [x.split(".")[0] for x in filenames]

datatest = pd.DataFrame({
    "id": labeltest,
     "filename": filenames})
datatest.head()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
plt.figure(figsize=(20,20)) # specifying the overall grid size
plt.subplots_adjust(hspace=0.4)


for i in range(10):
    
    plt.subplot(1,10,i+1)    # the number of images in the grid is 10*10 (100)
    filename = 'train/' + 'cat.' + str(i) + '.jpg'
    image = imread(filename)
    plt.imshow(image)
    plt.title('Cat',fontsize=12)
    plt.axis('off')

plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X=data['filename']
y=data['label']
X_train,X_val=train_test_split(data,random_state=42,stratify=y,test_size=0.3)

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_idg = datagen.flow_from_dataframe(X_train, "train/", x_col= "filename", y_col= "label",
                                    batch_size = 64,
                                    target_size=(150,150),class_mode='binary')
val_idg = datagen.flow_from_dataframe(X_val,"train/", x_col= "filename", y_col= "label",
                                    batch_size = 64,
                                    target_size=(150,150),class_mode='binary')
test_gen = test_datagen.flow_from_dataframe(datatest,"test1/",x_col= "filename", y_col= None, 
                                          class_mode=None,
                                          batch_size = 64,
                                          target_size = (150,150),
                                          shuffle = False
                                         )
test_gen

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, InputLayer, Dropout, GlobalAveragePooling2D
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                 input_shape=(150,150,3))
conv_base.trainable=False

In [ ]:
x = conv_base.output
x = GlobalAveragePooling2D()(x)
x=Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x=Dropout(0.25)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=conv_base.input, outputs=predictions)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    metrics=['accuracy'],
    loss='binary_crossentropy'
)
model.summary()

In [ ]:
model.fit(train_idg,epochs=10,batch_size=64,validation_data=val_idg)

In [ ]:
test_pred = model.predict(test_gen)
test_pred

In [ ]:
y_pred_submit = (test_pred >= 0.5).astype(float)
y_test_submit=y_pred_submit.reshape(1,-1).squeeze()
y_test_submit

In [ ]:
result_id=datatest['id']
result = pd.DataFrame({
    'id': result_id,
    'label': y_test_submit
})

In [ ]:
result.to_csv("submission.csv", index=False)